In this notebook we're going to look at hashes and salting those hashes, and using more complex salt than just a single random string. We're going to use a series of strings that each salt/hash iteration will influence the next salt that is used. 

From what I've read these would be good sorts of functions to use to store passwords in databases where you don't actually store the password itself, you store a hash of it. However, if you just hashed it, an attacker would have an easy time to create a rainbow table to reference and figure out what a password is. Iterate by adding salt (some string) a few hundred/thousand/million times and it becomes more difficult to guess. I figure, change the salt out each time, becomes even more hard to guess, just need to keep the salt a secret. Really these functions have not been vetted by any cryptographic expert at all, I just wanted to play with this sort of logic.

To do this, we will need keys, I intend to use `hexdigest` functions and `base64` functions, which will need to use the whole alphabet and all 10 digits. So the easiest way to get all those is with the `string` library built into python. 

In [1]:
import string

all_chars = string.ascii_letters + string.digits
L_chars = list(all_chars)
print(L_chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


Now that I have a list of characters, I can combine those into arbitrary length keys for my salt. In order to do that, I use a `while` loop and a counter to get up to length of the salt key I want, instead of having a bunch of `if` statements for how many `for _ in range` loops.  

Once the while loop has finished creating our list of keys, we can make those items the keys and generate the salt. We will create random salt using the `secrets` built-in library. In this library there is the `token_urlsafe` function which creates random strings of letters, numbers, and `-_` characters. 

In [2]:
from secrets import token_urlsafe as tus

salt_key_depth = 3 #skd

def salt_lib_gen(source_chars, skd=2, sl=256):
    #sl is Salt Length
    salt_dict = {}
    key_list = []
    c = 0
    while c < skd:
        if len(key_list) == 0:
            key_list = source_chars
            c += 1
        else:
            new_key_list = []
            for s in source_chars:
                for k in key_list:
                    new_key_list.append(f"{s}{k}")
            key_list = new_key_list
            c += 1
    for k in key_list:
        salt_dict[k] = tus(sl)
    return(salt_dict)

salt_lib = salt_lib_gen(L_chars, salt_key_depth)

Now that we have generated our salt dictionary, we can use that to take a string, and hash and salt it a bunch of times, grabbing the beginning of the string each time to determine the next salt to use. 

In [3]:
import hashlib

def hashbrowns(in_string, salt_d, salt_repetitions=10000, hasher=hashlib.sha256):
    pstring = in_string
    for _ in range(salt_repetitions):
        h = hasher()
        f3 = pstring.replace("_", "").replace("-", "")[:3]
        us = pstring + salt_d[f3]
        h.update(us.encode("utf8"))
        pstring = h.hexdigest()
    return(pstring)
    
hashbrowns('eggs_n_bacon', salt_lib, 1000000)

'63695bb30d792883e2799d1578d722518fef781b3c0a1eca11074ffb308dc27d'

Using just the `hexdigest` of the hasher will only give us the characters `0123456789abcdef`, that's not interesting enough and we created salt for the whole alphabet plus all of the digits. To make it more interesting let's encode each pass in base64, that way we can get the whole set of characters. 

In [4]:
import base64
def hashbrowns2(in_string, salt_d, salt_repetitions=10000, hasher=hashlib.sha256):
    pstring = in_string
    for _ in range(salt_repetitions):
        h = hasher()
        f3 = pstring.replace("_", "").replace("-", "")[:3]
        us = pstring + salt_d[f3]
        h.update(us.encode("utf8"))
        b64 = str(base64.b64encode(h.hexdigest().encode("utf8")), "utf8")
        pstring = b64
    return(pstring)

hashbrowns2('eggs_n_bacon', salt_lib, 1000000)

'OTRkNzJlOTNhMDMyZDYyNTdmYjBkMTVlMDFkYWM3NjJjNGVhODM5MzlmZjBlZTNkMjczMWVjN2Q2ZDk1YTJkYw=='

Still not as complex as it could be, let's use a character to to determine what three characters we grab to use as the key for the next salt to use. First, for consistency and reproduceability, we'll need a dictionary of random index places to use. 

In [5]:
import random

def char_usage_dict_gen(skd=2, str_range=16):
    char_usage_dict = {}
    for k in L_chars:
        char_usage_dict[k] = []
        for _ in range(skd):
            char_usage_dict[k].append(random.randint(0, str_range))
    return(char_usage_dict)

char_usage = char_usage_dict_gen(salt_key_depth)
print(char_usage)

{'a': [4, 12, 16], 'b': [15, 13, 6], 'c': [12, 5, 16], 'd': [2, 12, 16], 'e': [7, 6, 14], 'f': [1, 8, 9], 'g': [16, 8, 11], 'h': [0, 10, 13], 'i': [10, 11, 16], 'j': [13, 0, 9], 'k': [16, 3, 1], 'l': [6, 6, 0], 'm': [10, 13, 1], 'n': [4, 11, 16], 'o': [2, 10, 3], 'p': [14, 1, 9], 'q': [7, 4, 8], 'r': [1, 4, 9], 's': [15, 13, 7], 't': [8, 0, 6], 'u': [8, 9, 1], 'v': [10, 0, 12], 'w': [3, 15, 12], 'x': [7, 11, 1], 'y': [11, 7, 0], 'z': [1, 12, 2], 'A': [12, 1, 2], 'B': [6, 2, 5], 'C': [2, 12, 15], 'D': [15, 10, 0], 'E': [3, 15, 13], 'F': [16, 0, 9], 'G': [3, 4, 16], 'H': [12, 2, 9], 'I': [5, 11, 0], 'J': [6, 3, 7], 'K': [7, 10, 15], 'L': [14, 11, 9], 'M': [13, 16, 4], 'N': [7, 7, 8], 'O': [10, 6, 8], 'P': [15, 12, 1], 'Q': [15, 11, 4], 'R': [2, 1, 5], 'S': [13, 16, 3], 'T': [5, 14, 14], 'U': [14, 5, 12], 'V': [7, 13, 10], 'W': [7, 9, 6], 'X': [3, 8, 6], 'Y': [15, 3, 10], 'Z': [12, 13, 12], '0': [7, 3, 0], '1': [12, 7, 1], '2': [12, 11, 14], '3': [5, 9, 14], '4': [12, 5, 15], '5': [4, 10,

Now that we have a dictionary to pull characters out of the hashed string, we can use those to influence the salt used next. 
Thing is, we can't guarantee the input string is going to be of a sufficient length, so on the first loop, just use the first three characters, after that, with `hexdigest` and `base64` we can assume a long enough length (if we use 16 or 32 as the maximum string length in the previous function). 

In [6]:
def hashbrowns3(in_string, salt_d, char_usage, salt_repetitions=10000, hasher=hashlib.sha256, f_index=0):
    pstring = in_string
    for c in range(salt_repetitions):
        h = hasher()
        if c == 0:
            f3 = pstring.replace("_", "").replace("-", "")[:3]
            us = pstring + salt_d[f3]
        else:
            f = pstring[f_index]
            next_key = ""
            for v in char_usage[f]:
                next_key += pstring[v]
            us = pstring + salt_d[next_key]
        h.update(us.encode("utf8"))
        b64 = str(base64.b64encode(h.hexdigest().encode("utf8")), "utf8")
        pstring = b64
    return(pstring)

hashbrowns3('eggs_n_bacon', salt_lib, char_usage, 1000000)

'N2M1ZDEyODY0MWQ0MTBkOGViMDk1OWRjZDRjNDBiMjkzZmJlZTg5YmIxNDBlM2I2MjRiNTQwMTljODcwMGYxZA=='

Future idea: we could also create a dictionary of the source characters and link a hashlib method to them, so you randomize your hasher each hash round as well for an extra level of hashing complexity. 